In [1]:
import numpy as np
import pandas as pd

# Входные данные

In [39]:
shell = 1 # рассчитываем оболочку или нет
R = 1.9 # радиус оболочки (np.inf - прямая пластина)
plate = 1
plate1 = 0

In [40]:
E = 7e10  # модуль Юнга
ro = 2700  # плотоность алюминия
nu = 0.33  # коэффицент Пуассона
G = E / (2 * (1 + nu))  # модуль сдвига
L = 13.5  # L1  
W = 1.5   # L2
h = 0.0015   # тодщина пластины
D = (E * h ** 3) / (12 * (1 - nu ** 2))  # жесткость из уравнения колебания
N_end = 20  # расчтеные моды
M_end = 20
q1_end = 0   # число шпангаутов
q2_end = 0   # число стрингеров
h_sh = 0.1 # высота шпангоута
h_st = 0.02 # высота стрингера
b_sh_st = 0.002 # толщина сечения стрингера и шпангоута
S = N_end * M_end

In [41]:
if shell == 0 and R < 10000:
    plate1 = 1
    plate = 0
elif shell == 1:
    W = 2 * np.pi * R
    plate = 0
    plate1 = 0

# Расчет характеристик стрингеров и шпангоутов

In [42]:
S_sh = h_sh * b_sh_st  # профиль шпангоута
S_st = h_st * b_sh_st # профиль стрингера
I_sh = b_sh_st * (h_sh ** 3) / 12  # моменты инерции относительно оси y (толщина сечения)
I_st = b_sh_st * (h_st ** 3) / 12
# моменты кручения
Ik_sh = ((b_sh_st ** 3) * h_sh / 3) * (1 - 0.63 * (b_sh_st / h_sh) + 0.052 * (b_sh_st / h_sh) ** 5)
Ik_st = ((b_sh_st ** 3) * h_st / 3) * (1 - 0.63 * (b_sh_st / h_st) + 0.052 * (b_sh_st / h_st) ** 5)
# полярные моменты
Iro_sh = 0.5 * (b_sh_st * h_sh) * (h_sh ** 2 + b_sh_st ** 2)
Iro_st = 0.5 * (b_sh_st * h_st) * (h_st ** 2 + b_sh_st ** 2)
# погонные плотности шпангаутов и стрингеров (на единицу длины пластины)
ma_sh = ro * S_sh
ma_st = ro * S_st

In [43]:
# геометрия стрингеров и шпангоутов
b_pan_sh = L / (q1_end + 1) # пролет шпангаута
b_pan_st = W / (q2_end + 1) # пролет стрингера
S_pan_sh = b_pan_sh * h
S_pan_st = b_pan_st * h
V_sh = S_pan_sh * (h / 2 + h_sh / 2) / (S_pan_sh + S_sh)
V_st = S_pan_st * (h / 2 + h_st / 2) / (S_pan_st + S_st)

In [44]:
# координаты стрингеров и шпангоутов
x1 = np.arange(1, q1_end + 1) * b_pan_sh
x2 = np.arange(1, q2_end + 1) * b_pan_st

# Задаем пластину

In [45]:
# функции для вычисления ki, kj
def ku1(i):
    return (np.pi * i) / L
def ku2(j):
    return plate * (np.pi * j) / W + plate1 * (np.pi * j) / W + shell * j / R

In [46]:
# задаем матрицу оператора d^4/dx^4
Z0_eig = np.zeros((S, S))
for i in range(1, M_end + 1):
    for j in range(1 ,N_end + 1):
        k = (i - 1) * N_end + j
        ku = ku1(i) ** 4 + 2 * (ku1(i) * ku2(j)) ** 2 + ku2(j) ** 4
        Z0_eig[k-1][k-1] = D * ku + (plate1 * E * h * ku1(i) ** 4) / (ku * R ** 2) +(shell * E * h * ku1(i) ** 4) / (ku * R ** 2)

In [47]:
# задаем массовую матрицу
Z0_eig_m = np.diag([ro * h] * S)

In [48]:
# матрицы интегралов
V1 = np.diag([L / 2] * M_end)
V2 = np.diag([W / 2] * N_end)

# Матрицы для шпангоутов

In [49]:
sh10 = np.zeros((S, S))
sh20 = np.zeros((S, S))
for m1 in range(1, M_end + 1):
    for n1 in range(1, N_end + 1):
        for m2 in range(1, M_end + 1):
            for n2 in range(1, N_end + 1):
                j1 = (m1 - 1) * N_end + n1
                j2 = (m2 - 1) * N_end + n2
                Sum_r10_1, Sum_r20_1, Sum_r10_2, Sum_r20_2 = [0] * 4
                sh10_0 = E * (I_sh + S_sh * V_sh ** 2) * ku2(n1) ** 4
                sh20_0 = G * Ik_sh * (ku1(m1) * ku2(n1)) ** 2
                for q1 in x1:
                    # упругая матрица
                    Sum_r10_1 += (4 / (L * W)) * sh10_0 * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    Sum_r10_2 += (4 / (L * W)) * sh20_0 * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    # массовая матрица
                    Sum_r20_1 += (4 / (L * W)) * ma_sh * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    Sum_r20_2 += (4 / (L * W)) * ro * Iro_sh * (ku1(m1) ** 2) * np.sin(ku1(m1) * q1) * np.sin(ku1(m2) * q1) * V2[n1-1][n2-1]
                    
                sh10[j1-1][j2-1] = Sum_r10_1 + Sum_r10_2  # упругая
                sh20[j1-1][j2-1] = Sum_r20_1 + Sum_r20_2  # массовая

# Матрицы для стрингеров

In [50]:
st10 = np.zeros((S, S))
st20 = np.zeros((S, S))
for m1 in range(1, M_end + 1):
    for m2 in range(1, M_end + 1):
        for n1 in range(1, N_end + 1):
            for n2 in range(1, N_end + 1):
                j1 = (m1 - 1) * N_end + n1
                j2 = (m2 - 1) * N_end + n2
                Sum_r10_1, Sum_r20_1, Sum_r10_2, Sum_r20_2 = [0] * 4
                st10_0 = E * (I_st + S_st * V_st ** 2) * ku1(m1) ** 4
                st20_0 = G * Ik_st * (ku1(m1) * ku2(n1)) ** 2
                for q2 in x2:
                    # упругая матрица
                    slag1 = plate * np.sin(ku2(n1) * q2) * np.sin(ku2(n2) * q2) + shell * np.cos(ku2(n1) * q2) * np.cos(ku2(n2) * q2)
                    Sum_r10_1 += (4 / (L * W)) * st10_0 * V1[m1-1][m2-1] * slag1
                    slag2 = plate * np.cos(ku2(n1) * q2) * np.cos(ku2(n2) * q2) + shell * np.sin(ku2(n1) * q2) * np.sin(ku2(n2) * q2)
                    Sum_r10_2 += (4 / (L * W)) * st20_0 * ku2(n1) * ku2(n2) * V1[m1-1][m2-1] * slag2
                    # массовая матрица
                    Sum_r20_1 += (4 / (L * W)) * ma_st * V1[m1-1][m2-1] * slag1
                    Sum_r20_2 += (4 / (L * W)) * ro * Iro_st * V1[m1-1][m2-1] * slag2
                    
                st10[j1-1][j2-1] = Sum_r10_1 + Sum_r10_2  # упругая
                st20[j1-1][j2-1] = Sum_r20_1 + Sum_r20_2  # массовая

In [51]:
# упругая суммарная
Z0_eig_1 = Z0_eig + sh10 + st10
# массовая суммарная
Z0_eig_2 = Z0_eig_m + sh20 + st20

In [52]:
# финальная матрица
Z0_eig_final = np.linalg.inv(Z0_eig_2).T @ Z0_eig_1

In [53]:
# считаем собственные частоты
A, B = np.linalg.eig(Z0_eig_final)
eig_f = np.sqrt(np.sort(A)).reshape(N_end, M_end)
pd.DataFrame(eig_f / (np.pi * 2))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4.205139,4.381574,5.341765,5.411835,6.736715,8.423662,8.426454,8.435896,9.117019,9.356775,9.826934,10.350799,10.892588,12.443706,12.498645,12.813306,12.835671,13.206096,13.269602,14.041301
1,14.859298,15.085365,15.680961,15.867561,16.699939,16.736828,17.429300,17.592746,17.656330,18.070007,18.124256,19.344871,19.878897,19.949188,20.207035,20.250966,20.333720,20.714160,20.762211,21.029294
2,21.041944,21.548009,21.571338,22.619308,23.142485,23.191698,23.296016,23.583145,24.207711,24.886692,25.236237,25.349185,25.606281,25.831879,26.149906,26.382869,26.473137,26.701105,26.897205,27.170634
3,27.174984,28.027099,28.178923,29.036244,29.352879,29.391355,29.779605,29.780320,29.861506,30.051009,30.132052,30.425581,30.948842,31.080582,31.082203,31.377366,32.125668,33.210879,33.383921,33.406592
4,33.459085,33.622985,33.652870,33.930871,34.050787,34.055969,34.111077,34.333429,34.454073,34.838206,35.272954,35.280766,35.737660,36.467580,37.193596,37.264677,37.411164,37.427602,37.436065,37.673424
5,37.713196,38.104341,38.107532,38.421149,38.630773,38.765177,38.955494,39.079443,39.719976,40.243266,41.029044,41.200095,41.209299,41.277554,41.411964,41.482016,41.642443,41.710212,42.008594,42.268204
6,42.295280,42.387115,42.557181,42.742557,42.884193,42.901303,43.338702,44.403249,44.598799,44.895641,45.347860,45.583068,45.796124,46.060823,46.084148,46.160828,47.505941,47.518561,47.553875,47.978495
7,48.297185,48.824675,49.701454,49.774041,49.998204,50.573931,51.156317,51.199129,51.330046,51.397151,52.250955,53.692280,53.876474,53.938597,54.085396,54.121483,54.903116,55.201974,56.000936,56.042673
8,57.120069,57.873880,58.015050,58.327709,58.536496,58.543245,59.514082,60.240970,60.731258,61.065553,62.254997,62.330308,62.605855,63.260179,63.522126,64.196969,65.890320,65.954889,66.311483,66.994545
9,67.729582,68.848875,69.746975,69.748605,69.752222,69.755665,69.774672,69.814731,69.887518,70.007218,70.190432,70.456027,70.683281,70.824903,71.319684,71.964310,71.969591,71.990349,72.783537,73.802354


In [54]:
idx = A.argsort()   
A = A[idx]
B = B[:,idx]

# Рисуем решение

In [55]:
x = np.linspace(0, L, 1000)
y = np.linspace(0, W, 1000)
xx, yy = np.meshgrid(x, y)

In [61]:
mode = 2
w = np.sqrt(A[mode - 1]) / (np.pi * 2)

In [62]:
b = B[:, mode-1] # eigen vector

In [63]:
s = lambda i, j: (i - 1) * N_end + j - 1
zz1 = plate * np.array([[b[s(i, j)] * np.sin(ku1(i) * xx) * np.sin(ku2(j) * yy) for i in range(1, M_end + 1)] for j in range(1, N_end + 1)])
zz2 = shell * np.array([[b[s(i, j)] * np.sin(ku1(i) * xx) * np.cos(j * yy / R) for i in range(1, M_end + 1)] for j in range(1, N_end + 1)])

zz = zz1 + zz2

In [64]:
zz = np.sum(zz, (0, 1))

In [65]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib notebook
# Рисуем моду
fig = plt.figure()
ax = fig.add_subplot(111)
ax.contour(xx, yy, zz * (-1), cmap=cm.coolwarm)

<IPython.core.display.Javascript object>

In [29]:
# visualisation
fps = 10 # frame per sec
frn = 50 # frame number of the animation

In [32]:
t = np.linspace(0, 4 * np.pi / w, frn)
z_arr = np.array([zz * np.cos(w * i + np.pi / 2) for i in t])

In [33]:
def update_plot(frame_number, zarray, plot):
    plot[0].remove()
    plot[0] = ax.plot_surface(xx, yy, zarray[frame_number], cmap="magma")

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot = [ax.plot_surface(xx, yy, z_arr[0], color='0.75', rstride=1, cstride=1)]
ax.set_zlim(0,1.1)
ani = animation.FuncAnimation(fig, update_plot, frn, fargs=(z_arr, plot), interval=1000/fps)

<IPython.core.display.Javascript object>

NameError: name 'animation' is not defined

In [39]:
fn = '3x4mode'
ani.save(fn+'.gif',writer='imagemagick',fps=fps)